In [1]:
import os
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
import unidecode
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader 
from transformers import DistilBertTokenizer, DistilBertModel

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [13]:
path = '/home/Kaggle-challenge/workspace/data/'


train_d = pd.read_csv(path + 'train.csv')
# train_d = pd.read_csv('make_train.csv')
test = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submissions.csv')

In [14]:
train_na = train_d.dropna()

In [21]:
train_na

,id,comment_text,toxicity,severe_toxicity,obscene,threat,insult,identity_attack,sexual_explicit,female,male,christian,white,muslim,black,homosexual_gay_or_lesbian,jewish,psychiatric_or_mental_illness,asian
2,2,Haven't been here when the oil economy dead ha...,0.2,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,3,'\nBaloney.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
11,11,"Well, when you consider the success of Playboy...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
18,18,One more try at this for you and Trid. 4000 y...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
22,22,You seem to be ignorant of the fact that the I...,0.6,0.0,0.0,0.0,0.6,0.2,0.0,0.4,0.000000,0.0,0.000000,0.9,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068578,1068578,Don't expect the Alaskan budget problem to go ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1068579,1068579,"And two priests already on this thread, from w...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1068580,1068580,"Ken_Conklin, You have your views about churche...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1068583,1068583,"""The problem appears real, and not confined to...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
train_d_excluded = train_d[~train_d.index.isin(train_na.index)]
train_d_excluded

,id,comment_text,toxicity,severe_toxicity,obscene,threat,insult,identity_attack,sexual_explicit,female,male,christian,white,muslim,black,homosexual_gay_or_lesbian,jewish,psychiatric_or_mental_illness,asian
0,0,"No, we need Trump to stay in so he can lose.",0.166667,0.0,0.0,0.0,0.166667,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Wow. Precious.,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"When it hit and run, then the word ""accident"" ...",0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Baloney.\nThese were GOP that wanted to vote i...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,"China is building ""railways in Africa"" ...yep ...",0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068575,1068575,Just so long as the policemen in your argument...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1068576,1068576,"Wow, not a mention of 8 long years of Obama's ...",0.166667,0.0,0.0,0.0,0.166667,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1068577,1068577,Look at American history and one will see that...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1068581,1068581,"Too lenient. Considering her crimes, Mansfield...",0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
len(train_d)

1068585

In [22]:
749615 + 318970

1068585

In [26]:
IDENTITY_COLS = ['female',
       'male', 'christian', 'white', 'muslim', 'black',
       'homosexual_gay_or_lesbian', 'jewish', 'psychiatric_or_mental_illness',
       'asian']

In [27]:
train_d_excluded = train_d_excluded[IDENTITY_COLS]
train_d_excluded

,female,male,christian,white,muslim,black,homosexual_gay_or_lesbian,jewish,psychiatric_or_mental_illness,asian
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1068575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1068576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1068577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1068581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# subgroup = train_d[(train_d['female'] >= 0.5) | (train_d['male'] >= 0.5) | (train_d['christian'] >= 0.5) | (train_d['white'] >= 0.5) | (train_d['muslim'] >= 0.5) | (train_d['black'] >= 0.5) | (train_d['homosexual_gay_or_lesbian'] >= 0.5) | (train_d['jewish'] >= 0.5) | (train_d['psychiatric_or_mental_illness'] >= 0.5) | (train_d['asian'] >= 0.5)]
# SN = subgroup[subgroup['toxicity'] < 0.5]

In [53]:
# Background
# train_d = train_d[~((train_d['female'] >= 0.5) | (train_d['male'] >= 0.5) | (train_d['christian'] >= 0.5) | (train_d['white'] >= 0.5) | (train_d['muslim'] >= 0.5) | (train_d['black'] >= 0.5) | (train_d['homosexual_gay_or_lesbian'] >= 0.5) | (train_d['jewish'] >= 0.5) | (train_d['psychiatric_or_mental_illness'] >= 0.5) | (train_d['asian'] >= 0.5))]

In [6]:
# train_d = pd.concat([train_d, SN],axis=0)
# train_d.head()

In [6]:
train_d.info()

<class 'pandas.core.frame.DataFrame'>
Index: 937492 entries, 0 to 1068583
Data columns (total 19 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   id                             937492 non-null  int64  
 1   comment_text                   937492 non-null  object 
 2   toxicity                       937492 non-null  float64
 3   severe_toxicity                937492 non-null  float64
 4   obscene                        937492 non-null  float64
 5   threat                         937492 non-null  float64
 6   insult                         937492 non-null  float64
 7   identity_attack                937492 non-null  float64
 8   sexual_explicit                937492 non-null  float64
 9   female                         187877 non-null  float64
 10  male                           187877 non-null  float64
 11  christian                      187877 non-null  float64
 12  white                          187

In [5]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 32 #16 
EPOCHS = 2 # 1 
LEARNING_RATE = 1e-05
NUM_WORKERS = 4 # 2 

In [6]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', string)
    text = re.sub("[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]", " ", text)
    return text

def preprocess(df, text_column="comment_text"):
    url_pattern = r"https?://\S+|www\.\S+"
    # remove url
    df[text_column] = df[text_column].str.replace(url_pattern, " ")

    # apply unidecode
    df[text_column] = df[text_column].map(unidecode.unidecode)
    
    # remove emoji
    df[text_column] = df[text_column].map(remove_emoji)

    # apply lower
    df[text_column] = df[text_column].str.lower()
    
    return df

train_d = preprocess(train_d)

In [10]:
# train =  train[['comment_text','toxicity']]
# train['target'] = (train['toxicity'] >= 0.5).astype(int)

In [7]:
col = ['toxicity', 'severe_toxicity', 'obscene',
       'threat', 'insult', 'identity_attack', 'sexual_explicit']
value_train = train_d[col]

In [8]:
value_train.iloc[:]

,toxicity,severe_toxicity,obscene,threat,insult,identity_attack,sexual_explicit
0,0.166667,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.200000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
1068580,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1068581,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1068582,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1068583,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [8]:
# col = ['toxicity', 'severe_toxicity', 'obscene',
#        'threat', 'insult', 'identity_attack', 'sexual_explicit']

for i in col:
    value_train[i] = (value_train[i] >= 0.5).astype(int)

train_d['target'] = value_train.iloc[:, :].values.tolist()


/tmp/ipykernel_951481/329998477.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value_train[i] = (value_train[i] >= 0.5).astype(int)
/tmp/ipykernel_951481/329998477.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value_train[i] = (value_train[i] >= 0.5).astype(int)
/tmp/ipykernel_951481/329998477.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

In [9]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len: int, eval_mode: bool = False):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.text = dataframe.comment_text
        self.eval_mode = eval_mode 
        if self.eval_mode is False:
            self.targets = self.data.target
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text.iloc[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        output = {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
        }
                
        if self.eval_mode is False:
            output['targets'] = torch.tensor(self.targets.iloc[index], dtype=torch.float)
                
        return output

In [10]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)
training_set = MultiLabelDataset(train_d, tokenizer, MAX_LEN)

In [13]:
training_set[0]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/root/anaconda3/envs/py31/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


{'ids': tensor([ 101, 2053, 2057, 2342, 8398, 2000, 2994, 1999, 2061, 2002, 2064, 4558,
          102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0, 

In [11]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': NUM_WORKERS
                }
training_loader = DataLoader(training_set, **train_params)

In [12]:
class DistilBERTClass(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 7)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [13]:
model = DistilBERTClass()
model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DistilBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in

In [14]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [15]:
def train(epoch):
    
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        loss.backward()
        optimizer.step()

In [16]:
for epoch in range(EPOCHS):
    train(epoch)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/root/anaconda3/envs/py31/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific valu

Epoch: 0, Loss:  0.716808021068573


5000it [37:26,  2.27it/s]

Epoch: 0, Loss:  0.055100105702877045


10000it [1:14:08,  2.27it/s]

...

25000it [3:03:38,  2.27it/s]

Epoch: 1, Loss:  0.08558698743581772


30000it [3:40:22,  2.27it/s]

Epoch: 1, Loss:  0.05465519428253174


33394it [4:05:17,  2.27it/s]


## SUBGROUP active learning

In [26]:
test

,id,comment_text
0,0,Not me - I've seen investigations into trainin...
1,1,She was fully deportable and I am glad someone...
2,2,That's for the courts to decide.
3,3,"When she was a scrawny 11-year-old, Sherry Joh..."
4,4,I cannot help but wonder what Mueller is findi...
...,...,...
431410,431410,What would you propose France should do to rid...
431411,431411,"The article was about Hoodoo, and the criteria..."
431412,431412,"Apparently, according to numerous nationwide w..."
431413,431413,Obama on the United States as a Christian nati...


In [26]:
subgroup = subgroup[['id', 'comment_text']]

In [27]:
subgroup_set = MultiLabelDataset(subgroup, tokenizer, MAX_LEN, eval_mode = True)
testing_params = {'batch_size': TRAIN_BATCH_SIZE,
               'shuffle': False,
               'num_workers': 2
                }
subgroup_loader = DataLoader(subgroup_set, **testing_params)

In [28]:
all_subgroup_pred = []

def sub_t(epoch):
    model.eval()
    
    with torch.inference_mode():
    
        for _, data in tqdm(enumerate(subgroup_loader, 0)):


            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            outputs = model(ids, mask, token_type_ids)
            probas = torch.sigmoid(outputs)

            all_subgroup_pred.append(probas)
    return probas

In [ ]:
subgroup

In [29]:
probas = sub_t(model)

0it [00:00, ?it/s]/root/anaconda3/envs/py31/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/root/anaconda3/envs/py31/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave

In [30]:
probas

tensor([[5.4017e-03, 6.0077e-08, 6.7181e-05, 1.5189e-04, 2.8324e-03, 1.6648e-04,
         1.6031e-04],
        [3.5383e-03, 1.1021e-07, 1.3193e-04, 4.9015e-04, 4.6056e-04, 7.5086e-05,
         1.7743e-04],
        [1.8004e-02, 3.7344e-07, 1.3199e-04, 2.9430e-04, 6.4860e-03, 2.5748e-03,
         1.8874e-04],
        [6.3338e-02, 9.7932e-08, 5.4433e-04, 1.8328e-04, 6.0089e-02, 2.9282e-04,
         3.4576e-04],
        [4.5931e-02, 1.2609e-06, 1.6100e-04, 3.2304e-04, 5.6940e-03, 1.9249e-02,
         1.8347e-04]], device='cuda:0')

In [33]:
all_subgroup_pred = torch.cat(all_subgroup_pred)

In [37]:
all_subgroup_pred = all_subgroup_pred.cpu()

In [38]:
all_subgroup_pred 

tensor([[1.5704e-02, 2.6573e-07, 4.4558e-04,  ..., 2.3416e-03, 2.2214e-04,
         1.7254e-02],
        [5.4540e-01, 1.8683e-06, 1.2340e-03,  ..., 4.7111e-01, 1.4231e-02,
         5.7379e-04],
        [7.6797e-03, 5.2629e-07, 9.7408e-05,  ..., 8.7107e-04, 3.7594e-03,
         1.5708e-04],
        ...,
        [1.8004e-02, 3.7344e-07, 1.3199e-04,  ..., 6.4860e-03, 2.5748e-03,
         1.8874e-04],
        [6.3338e-02, 9.7932e-08, 5.4433e-04,  ..., 6.0089e-02, 2.9282e-04,
         3.4576e-04],
        [4.5931e-02, 1.2609e-06, 1.6100e-04,  ..., 5.6940e-03, 1.9249e-02,
         1.8347e-04]])

In [41]:
col = ['toxicity', 'severe_toxicity', 'obscene',
       'threat', 'insult', 'identity_attack', 'sexual_explicit']
for i,name in enumerate(col):
    subgroup[name] = all_subgroup_pred[:, i]

In [42]:
subgroup

,id,comment_text,toxicity,severe_toxicity,obscene,threat,insult,identity_attack,sexual_explicit
11,11,"Well, when you consider the success of Playboy...",0.015704,2.657304e-07,0.000446,0.000147,0.002342,0.000222,0.017254
22,22,You seem to be ignorant of the fact that the I...,0.545399,1.868301e-06,0.001234,0.000561,0.471114,0.014231,0.000574
37,37,What do Christians have to do with this articl...,0.007680,5.262926e-07,0.000097,0.000148,0.000871,0.003759,0.000157
59,59,Hillary will meet the requirements of her fund...,0.000512,1.169475e-07,0.000085,0.000058,0.000294,0.000052,0.000090
66,66,"Many couples incorporate the above mentioned ""...",0.009281,4.219486e-07,0.000433,0.000190,0.001041,0.000182,0.021162
...,...,...,...,...,...,...,...,...,...
1068545,1068545,"In a recent highly publicised case, the man in...",0.005402,6.007705e-08,0.000067,0.000152,0.002832,0.000166,0.000160
1068546,1068546,https://www.youtube.com/watch?v=Qup47XJGIl4\n\...,0.003538,1.102081e-07,0.000132,0.000490,0.000461,0.000075,0.000177
1068556,1068556,Liu Xiaobo is too famous. If he was able to tr...,0.018004,3.734395e-07,0.000132,0.000294,0.006486,0.002575,0.000189
1068572,1068572,Well aren't you our Debbie Downer? You must be...,0.063338,9.793220e-08,0.000544,0.000183,0.060089,0.000293,0.000346


In [57]:
result = pd.concat([train_d,subgroup],axis=0)

result

,id,comment_text,toxicity,severe_toxicity,obscene,threat,insult,identity_attack,sexual_explicit,female,male,christian,white,muslim,black,homosexual_gay_or_lesbian,jewish,psychiatric_or_mental_illness,asian
0,0,"No, we need Trump to stay in so he can lose.",0.166667,0.000000e+00,0.000000,0.000000,0.166667,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Wow. Precious.,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Haven't been here when the oil economy dead ha...,0.200000,0.000000e+00,0.000000,0.200000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,'\nBaloney.,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,"When it hit and run, then the word ""accident"" ...",0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068545,1068545,"In a recent highly publicised case, the man in...",0.005402,6.007705e-08,0.000067,0.000152,0.002832,0.000166,0.000160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1068546,1068546,https://www.youtube.com/watch?v=Qup47XJGIl4\n\...,0.003538,1.102081e-07,0.000132,0.000490,0.000461,0.000075,0.000177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1068556,1068556,Liu Xiaobo is too famous. If he was able to tr...,0.018004,3.734395e-07,0.000132,0.000294,0.006486,0.002575,0.000189,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1068572,1068572,Well aren't you our Debbie Downer? You must be...,0.063338,9.793220e-08,0.000544,0.000183,0.060089,0.000293,0.000346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
result = result.sort_index()

In [66]:
result.to_csv("make_train.csv", index=False)

# TEST

In [17]:
test_set = MultiLabelDataset(test, tokenizer, MAX_LEN, eval_mode = True)
testing_params = {'batch_size': TRAIN_BATCH_SIZE,
               'shuffle': False,
               'num_workers': 4
                }
test_loader = DataLoader(test_set, **testing_params)

In [18]:
all_test_pred = []

def test(epoch):
    model.eval()
    
    with torch.inference_mode():
    
        for _, data in tqdm(enumerate(test_loader, 0)):


            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            outputs = model(ids, mask, token_type_ids)
            probas = torch.sigmoid(outputs)

            all_test_pred.append(probas)
    return probas

In [19]:
probas = test(model)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of

In [20]:
all_test_pred = torch.cat(all_test_pred)

In [21]:
submission['prediction'] = all_test_pred[:,0].cpu()

In [22]:
submission

,id,prediction
0,0,0.000596
1,1,0.003322
2,2,0.000332
3,3,0.058943
4,4,0.085496
...,...,...
431410,431410,0.192491
431411,431411,0.000388
431412,431412,0.000331
431413,431413,0.012632


In [23]:
submission.to_csv('submit_Epoch2.csv', index=False)

In [36]:
sub = submission.drop(columns='prediction')
sub

,id
0,0
1,1
2,2
3,3
4,4
...,...
431410,431410
431411,431411
431412,431412
431413,431413


In [33]:
for i, name in enumerate(col):
    sub[name] = all_test_pred[:,i].cpu()

In [34]:
sub.to_csv('subtype.csv',index=False)